# Define Spatial Extent of Project

Metropolitan Council NAD83 -- UTM zone 15N -- epsg:26915

Source: https://gisdata.mn.gov/dataset/us-mn-state-metc-bdry-census2010counties-ctus

Metadata: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/metadata/metadata.html


In [1]:
# Import libraries

import os # For working with Operating System
import shutil # For deleting folders

import urllib # For accessing websites
import zipfile # For extracting from Zipfiles

import geopandas as gpd # Spatial Data Manipulation


In [2]:
# Download from source

# Retrieve file (~ 5mb)

urllib.request.urlretrieve("https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/shp_bdry_census2010counties_ctus.zip",
                           "shp_bdry_census2010counties_ctus.zip") 
# Extract from Zip

with zipfile.ZipFile("shp_bdry_census2010counties_ctus.zip","r") as zip_ref:
    zip_ref.extractall()
    zip_ref.close()

In [3]:
# Read, Select, and Save Data

metro = gpd.read_file(r"Census2010CTUs.shp") # Read data to geodataframe

mpls = metro[metro['CTU_NAME']=='Minneapolis'] # Select Minneapolis' shape

mpls.to_file(r'mpls_boundary.shp') # Save file

In [4]:
# Remove downloaded files

cwd = os.getcwd() # Current Working Directory

for root, dirs, files in os.walk(cwd): # Loop over files & folders
    for file in files:
        if 'mpls' in file or file == 'DefineBoundary.ipynb': # Keep new shapefile & notebook
            pass
        else:
            os.remove(file) # Remove all others
    for folder in dirs:
        shutil.rmtree(cwd+ '\\' + folder) # Remove folders